# Libraries

In [ ]:
import pandas as pd
import numpy as np
import math
from math import log
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.metrics import f1_score
import sys

# Lettura

In [ ]:
#time interval used in our works
time_slices = {
    0: {
        'future_start': '2016-07-03',
        'future_end': '2016-08-02',
        'start' : '2016-06-03',
        'end': '2016-07-02'
    },
    
    1: {
        'future_start': '2016-08-03',
        'future_end': '2016-09-02',
        'start': '2016-07-03',
        'end': '2016-08-02'
    },
    
    2:{
        'future_start': '2016-09-03',
        'future_end': '2016-10-02',
        'start': '2016-08-03',
        'end': '2016-09-02'
    },
    
    3:{
        'future_start': '2016-10-03',
        'future_end': '2016-11-02',
        'start': '2016-09-03',
        'end': '2016-10-02'
    }
}

time_slices = {
    0: {
        'future_start': '2016-12-01',
        'future_end': '2016-12-31',
        'start' : '2016-11-01',
        'end': '2016-11-30'
    },
    
    1: {
        'future_start': '2017-01-01',
        'future_end': '2017-01-31',
        'start': '2016-12-01',
        'end': '2016-12-31'
    }
}

In [ ]:
PERIOD = 1
SINGLE_SLICE = False
INCREMENTAL_SETTING = False
LABEL_EXP = "AllUserIntervalAugust2016"
K = 30 #num_topic
ftest = 'train' if INCREMENTAL_SETTING else 'test'

In [ ]:
future_start = time_slices[PERIOD]['future_start']
future_end = time_slices[PERIOD]['future_end']
#start = time_slices[PERIOD]['start']
#end = time_slices[PERIOD]['end']

start_test = time_slices[PERIOD+1]['future_start']
end_test = time_slices[PERIOD+1]['future_end']

In [ ]:
ftrain = 'train' + ('_single' if SINGLE_SLICE else '')

In [ ]:
train = pd.read_csv("%s/sim_%s_%s_%s" % ('../dummy-data/traditional/',ftrain, future_start, future_end), index_col=0)

In [ ]:
train = train.dropna() #Double check isolates and self loops

In [ ]:
#len(train), len(train[train['ldaAll_0_from'] == 0]), len(train[train['ldaAll_0_to'] == 0])

In [ ]:
len(train[train['y']==0]),len(train[train['y']==1])

In [ ]:
#train = train.loc[~((train['ldaAll_0_from'] == 0) | (train['ldaAll_0_to'] == 0))]

In [ ]:
len(train)

In [ ]:
"""
struct_f = ['comm_neigh', 'jaccard_coef', 'adamic_adar', \
            'res_allocation', 'pref_attachment','pagerank_from','pagerank_to',\
            'neigh_degree_from','neigh_degree_to','katz_from','katz_to','lrw','in_degree_from','in_degree_to',\
           'out_degree_from','out_degree_to']

text_f = ['tag_jaccard_coef','avg_post_length_from','avg_post_length_to','std_post_length_from','std_post_length_to',\
          'num_post_from',\
       'num_post_to','num_comment_from','num_comment_to','num_tag_from','num_tag_to'] \
        + ['ldaAll_{}_from'.format(i) for i in range(K)] + ['ldaAll_{}_to'.format(i) for i in range(K)]\
        + ['ldaComment_{}_from'.format(i) for i in range(K)] + ['ldaComment_{}_to'.format(i) for i in range(K)]\
        + ['kl_ldaAll','kl_ldaComment']
        
""" #OLD FEATURES

In [ ]:
struct_f = ['comm_neigh', 'jaccard_coef', 'adamic_adar', \
            'res_allocation', 'pref_attachment','pagerank_from','pagerank_to',\
            'neigh_degree_from','neigh_degree_to','katz_from','katz_to','lrw','in_degree_from','in_degree_to',\
           'out_degree_from','out_degree_to']

text_f = ['tag_jaccard_coef','avg_post_length_from','avg_post_length_to','std_post_length_from','std_post_length_to',\
          'num_post_from',\
       'num_post_to','num_comment_from','num_comment_to','num_tag_from','num_tag_to'] \
        + ['euclidean_dist_all','cosine_dist_all','chebyshev_dist_all','jaccard_dist_all']\
        + ['euclidean_dist_comment','cosine_dist_comment','chebyshev_dist_comment','jaccard_dist_comment']

In [ ]:
#SOME CHECKS
#np.isinf(train[[col for col in train.columns if 'kl' in col]]).any()

In [ ]:
#np.isinf(train[[col for col in train.columns if 'kl' in col]]).sum()

In [ ]:
#train = train.drop(['kl_ldaAll','kl_ldaComment'],axis=1)

In [ ]:
scaler = StandardScaler()
#scaler = MinMaxScaler()
#LABEL_EXP += 'MinMax'
toScale = list(train.drop(['from','to','y'],axis=1).columns)
train[toScale] = scaler.fit_transform(train[toScale].values)

In [ ]:
#CHOOSE THE COMBINATION OF FEATURES

train = train[['from','to','y'] + text_f]
#train = train[['from','to','y'] + struct_f]
LABEL_EXP += 'Text'
#LABEL_EXP += 'Struct'
#feature_sel = [f for f in train.columns if 'to' not in f.lower() and 'from' not in f.lower() and 'y' not in f.lower()]
#train = train[['from','to','y'] + feature_sel]
#LABEL_EXP += 'StrictlySim'

In [ ]:
#LABEL_EXP += 'StructText'

In [ ]:
"""
LABEL_EXP = LABEL_EXP + "CommentFromSelectionClassic"
featureSel = [f for f in text_f if 'comment' in f.lower() and 'from' in f and 'num' not in f] + [f for f in text_f if 'lda' not in f.lower()]
train = train[['from','to','y']+ struct_f + featureSel]
"""

## Models 

In [ ]:
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.experimental import enable_hist_gradient_boosting  

from sklearn.ensemble import GradientBoostingClassifier

from sklearn.linear_model import LogisticRegression

from sklearn.neural_network import MLPClassifier

from sklearn.dummy import DummyClassifier

from sklearn import svm

from sklearn.naive_bayes import GaussianNB

from sklearn.linear_model import Perceptron

from sklearn.dummy import DummyClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(train.drop(['from','to','y'],axis=1), train[['y']], train_size = 0.75)

In [ ]:
test = pd.read_csv("../dummy-data/traditional/sim_%s_%s_%s" % (ftest, start_test, end_test), index_col=0)

In [ ]:
test = test.dropna()

In [ ]:
#len(test), len(test[test['ldaAll_0_from'] == 0]), len(test[test['ldaAll_0_to'] == 0])

In [ ]:
len(test[test['y']==0]),len(test[test['y']==1])

In [ ]:
#test = test.loc[~((test['ldaAll_0_from'] == 0) | (test['ldaAll_0_to'] == 0))]

In [ ]:
len(test)

In [ ]:
test = test[['from','to','y'] + struct_f + text_f ] #filter old dataset

In [ ]:
#np.isinf(test[[col for col in test.columns if 'kl' in col]]).any()

In [ ]:
#test = test.drop(['kl_ldaAll','kl_ldaComment'],axis=1)

In [ ]:
test[toScale] = scaler.fit_transform(test[toScale])

In [ ]:
#CHOOSE THE SAME CONFIG OF FEATURES OF TRAIN SET

#test = test.drop([f for f in text_f if f in test.columns],axis=1)
#test = test.drop(struct_f,axis=1)
#test = test.drop(text_to_f,axis=1)
#test = test[['from','to','y']+ struct_f]
test = test[['from','to','y']+ text_f]
#test = test[['from','to','y']+ feature_sel]

# Dummy

In [ ]:
dummy = DummyClassifier(strategy='uniform')
dummy.fit(x_train, y_train.values.ravel())

In [ ]:
print('Test')
dummy_pred = dummy.predict(test.drop(['from','to','y'],axis=1))
acc_dummy = accuracy_score(test['y'], dummy_pred)
f1_dummy = f1_score(test['y'],dummy_pred)
print(f1_dummy)
print(acc_dummy)
print(confusion_matrix(test['y'], dummy_pred))
print(classification_report(test['y'], dummy_pred))

###  Gradient boosting

In [ ]:
#it takes a bit
boost_cl = GradientBoostingClassifier(learning_rate=0.001,max_depth=5,n_estimators=1000)
boost_cl.fit(x_train, y_train.values.ravel())

In [ ]:
print('GB')
print('Train')
boost_pred = boost_cl.predict(x_train)
print(f1_score(y_train,boost_pred))
print(accuracy_score(y_train, boost_pred))
print(confusion_matrix(y_train, boost_pred))

In [ ]:
print('Val')
boost_pred = boost_cl.predict(x_test)
print(f1_score(y_test,boost_pred))
print(accuracy_score(y_test, boost_pred))
print(confusion_matrix(y_test, boost_pred))
print(classification_report(y_test, boost_pred))

In [ ]:
# On test set
print('Test')
boost_pred = boost_cl.predict(test.drop(['from','to','y'],axis=1))
acc_gradient= accuracy_score(test['y'], boost_pred)
f1_gradient = f1_score(test['y'],boost_pred)
print(f1_gradient)
print(acc_gradient)
print(confusion_matrix(test['y'], boost_pred))
print(classification_report(test['y'], boost_pred))

In [ ]:
#fare shuffle
#train_cv = shuffle(train)
#cv_results= cross_validate(boost_cl,train_cv.drop(['from','to','y'],axis=1), train_cv[['y']].values.ravel(), cv=5, scoring='accuracy')

In [ ]:
#print(cv_results['test_score'])
#print(np.mean(cv_results['test_score']))
#print(np.std(cv_results['test_score']))

In [ ]:
FILTER_TOP = 20

In [ ]:
import os
try:
    os.mkdir("results{}".format(LABEL_EXP))
except FileExistsError: pass

In [ ]:
import matplotlib.ticker as mtick

f_imp = pd.DataFrame({'feature':x_train.columns,\
                      'importance':boost_cl.feature_importances_})\
             .sort_values(by='importance',ascending=False)[:FILTER_TOP]
plt.figure(figsize=(10, 7))
plt.title("Gradient Boosting Feature importances")
plt.barh(range(FILTER_TOP), f_imp.importance, 0.5 ,color='#ffcc99', ec='#ff8000', align="center")
plt.yticks(range(FILTER_TOP),f_imp.feature)
plt.ylim([-1,FILTER_TOP])

plt.savefig("%s_%s_%s.pdf" % ('results{}/GB_acc'.format(LABEL_EXP),future_start,future_end),bbox_inches='tight') 
plt.show();

### SVM (features importance available only for linear kernel)

In [ ]:
svm_cl = svm.SVC(kernel='rbf',C=0.01)
svm_cl.fit(x_train,y_train.values.ravel())

In [ ]:
print('SVM')
print('Train')
svm_pred = svm_cl.predict(x_train)
print(f1_score(y_train,svm_pred))
print(accuracy_score(y_train,svm_pred))
print(confusion_matrix(y_train,svm_pred))

In [ ]:
print('Val')
svm_pred = svm_cl.predict(x_test)
print(f1_score(y_test,svm_pred))
print(accuracy_score(y_test,svm_pred))
print(confusion_matrix(y_test,svm_pred))
print(classification_report(y_test,svm_pred))

In [ ]:
print('Test')
svm_pred = svm_cl.predict(test.drop(['from','to','y'],axis=1))
acc_svm = accuracy_score(test['y'], svm_pred)
f1_svm = f1_score(test['y'],svm_pred)
print(f1_svm)
print(acc_svm)
print(confusion_matrix(test['y'], svm_pred))
print(classification_report(test['y'], svm_pred))

def f_importances(coef, names):
    imp = coef
    imp,names = zip(*sorted(zip(imp,names)))
    plt.barh(range(len(names)), imp, align='center')
    plt.yticks(range(len(names)), names)
    plt.show()

features_names = list(train.iloc[:,3:])
f_importances(svm_cl.coef_, features_names)

### Random Forest 

In [ ]:
ran_for = RandomForestClassifier(bootstrap=False,max_depth=10,max_features='sqrt',min_samples_leaf=4,min_samples_split=5,n_estimators=5)
ran_for.fit(x_train,y_train.values.ravel())

In [ ]:
print('RF')
print('Train')
rf_pred = ran_for.predict(x_train)
print(f1_score(y_train,rf_pred))
print(accuracy_score(y_train,rf_pred))
print(confusion_matrix(y_train, rf_pred))

In [ ]:
print('Val')
rf_pred = ran_for.predict(x_test)
print(f1_score(y_test,rf_pred))
print(accuracy_score(y_test,rf_pred))
print(confusion_matrix(y_test,rf_pred))
print(classification_report(y_test,rf_pred))

In [ ]:
print('Test')
rf_pred = ran_for.predict(test.drop(['from','to','y'],axis=1))
acc_rforest= accuracy_score(test['y'],rf_pred)
f1_rforest = f1_score(test['y'],rf_pred)
print(f1_rforest)
print(acc_rforest)
print(confusion_matrix(test['y'], rf_pred))
print(classification_report(test['y'], rf_pred))

In [ ]:
features_imp = pd.DataFrame({'feature':x_train.columns,'importance':ran_for.feature_importances_})\
                .sort_values(by='importance',ascending=False)[:FILTER_TOP]
plt.figure(figsize=(10, 7))
plt.title("Random Forest Feature importances")
plt.barh(range(FILTER_TOP), features_imp.importance, 0.5 ,color='#ffcc99', ec='#ff8000', align="center")
plt.yticks(range(FILTER_TOP),features_imp.feature)
plt.ylim([-1, FILTER_TOP])
   
plt.savefig("%s_%s_%s.pdf" % ('results{}/rfor_acc'.format(LABEL_EXP),future_start,future_end),bbox_inches='tight')

plt.show()

### Logistic Regression

In [ ]:
log_reg = LogisticRegression(solver='lbfgs',penalty='l2',C=1e-05)
log_reg.fit(x_train,y_train.values.ravel())

In [ ]:
print('LogReg')
print('Train')
lr_pred = log_reg.predict(x_train)
print(f1_score(y_train,lr_pred))
print(accuracy_score(y_train, lr_pred))
print(confusion_matrix(y_train, lr_pred))

In [ ]:
print('Val')
lr_pred = log_reg.predict(x_test)
print(f1_score(y_test,lr_pred))
print(accuracy_score(y_test,lr_pred))
print(confusion_matrix(y_test,lr_pred))
print(classification_report(y_test,lr_pred))

In [ ]:
print('Test')
lr_pred = log_reg.predict(test.drop(['from','to','y'],axis=1))
acc_logit = accuracy_score(test['y'], lr_pred)
f1_logit = f1_score(test['y'],lr_pred)
print(f1_logit)
print(acc_logit)
print(confusion_matrix(test['y'], lr_pred))
print(classification_report(test['y'], lr_pred))

In [ ]:
feat_imp = pd.DataFrame(x_train.columns, columns=['feature'])
feat_imp['importance'] = log_reg.coef_[0]
feat_imp = feat_imp.sort_values(by='importance', ascending=False)

In [ ]:
plt.figure(figsize=(10, 7))
plt.title("Logistic Regression Feature importances")
plt.barh(range(FILTER_TOP), feat_imp.importance[:FILTER_TOP], 0.5 ,color='#ffcc99', ec='#ff8000', align="center")
plt.yticks(range(FILTER_TOP),feat_imp.feature[:FILTER_TOP])
plt.ylim([-1, FILTER_TOP])


plt.savefig("%s_%s_%s.pdf" % ('results{}/logit_acc'.format(LABEL_EXP),future_start,future_end),bbox_inches='tight')
plt.show();

In [ ]:
from sklearn import metrics
y_pred_proba = log_reg.predict_proba(x_test)[::,1]
fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_proba)
auc = metrics.roc_auc_score(y_test, y_pred_proba)
plt.plot(fpr,tpr,label="data 1, auc="+str(auc))
plt.legend(loc=4)
plt.show()

### XGBOOST 

In [ ]:
import xgboost as xgb
from xgboost import XGBClassifier

In [ ]:
xg_class= XGBClassifier(objective='binary:logistic', eval_metric='logloss',gamma=5,learning_rate=0.001,max_depth=3,min_child_weight=1,n_estimators=1000)
xg_class.fit(x_train,y_train.values.ravel())

In [ ]:
print('XGB')
print('Train')
xg_pred = xg_class.predict(x_train)
print(f1_score(y_train,xg_pred))
print(accuracy_score(y_train,xg_pred))
print(confusion_matrix(y_train, xg_pred))
print(classification_report(y_train, xg_pred))

In [ ]:
print('Val')
xg_pred = xg_class.predict(x_test)
print(f1_score(y_test,xg_pred))
print(accuracy_score(y_test,xg_pred))
print(confusion_matrix(y_test, xg_pred))
print(classification_report(y_test, xg_pred))

In [ ]:
print('Test')
xg_pred = xg_class.predict(test.drop(['from','to','y'],axis=1))
acc_xgb = accuracy_score(test['y'], xg_pred)
f1_xgb = f1_score(test['y'],xg_pred)
print(f1_xgb)
print(acc_xgb)
print(confusion_matrix(test['y'], xg_pred))
print(classification_report(test['y'], xg_pred))

In [ ]:
xgb.plot_importance(xg_class)
plt.savefig("%s_%s_%s.pdf" % ('results{}/xgb_acc'.format(LABEL_EXP),future_start,future_end),bbox_inches='tight')

In [ ]:
xgb.plot_tree(xg_class, num_trees=4);

In [ ]:
sorted(list(xg_class.get_booster().get_score(importance_type="weight").items()),key=lambda x: x[1],reverse=True)[:20]

### MLP

In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=(16,),solver='sgd',alpha=1)
mlp.fit(x_train,y_train.values.ravel())

In [ ]:
print('MLP')
print('Train')
mlp_pred = mlp.predict(x_train)
print(f1_score(y_train,mlp_pred))
print(accuracy_score(y_train, mlp_pred))
print(confusion_matrix(y_train, mlp_pred))

In [ ]:
print('Val')
mlp_pred = mlp.predict(x_test)
print(f1_score(y_test,mlp_pred))
print(accuracy_score(y_test,mlp_pred))
print(confusion_matrix(y_test, mlp_pred))
print(classification_report(y_test, mlp_pred))

In [ ]:
print('Test')
mlp_pred = mlp.predict(test.drop(['from','to','y'],axis=1))
acc_mlp = accuracy_score(test['y'], mlp_pred)
f1_mlp = f1_score(test['y'],mlp_pred)
print(f1_mlp)
print(acc_mlp)
print(confusion_matrix(test['y'], mlp_pred))
print(classification_report(test['y'], mlp_pred))

In [ ]:
with open('results{}/results{}.txt'.format(LABEL_EXP,LABEL_EXP),'a+') as r:
    r.write(LABEL_EXP+'\n')
    r.write('F1 scores\n')
    r.write('GB: ' + str(f1_gradient) + '\n')
    r.write('SVM: ' + str(f1_svm) +'\n')
    r.write('NB: ' + str(f1_bayes) +'\n')
    r.write('RF: ' + str(f1_rforest)+'\n')
    r.write('LR: ' + str(f1_logit)+'\n')
    r.write('XGB: ' + str(f1_xgb)+'\n')
    r.write('MLP: '+str(f1_mlp)+'\n')
    r.close()